In [3]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.callbacks import EarlyStopping 
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV



In [4]:
# Importando os arquivos JSON
df = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\applicants.json").T
df_prospects = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\prospects.json").T
df_vagas = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\vagas.json").T

In [5]:
# "Abrindo" os dados de applicants
cols_com_dicionario =['infos_basicas','informacoes_pessoais','informacoes_profissionais','formacao_e_idiomas','cargo_atual']

dfs_normalizados = []

for col in cols_com_dicionario:
    df_temp = pd.json_normalize(df[col])
    df_temp.columns = [f"{col}_{subcol}" for subcol in df_temp.columns]
    dfs_normalizados.append(df_temp)

df_normalizado = pd.concat([df.drop(columns=cols_com_dicionario)] + dfs_normalizados, axis=1)
df_normalizado

,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
31000,assistente administrativo\n\n\nsantosbatista\n...,,,(19) 94846-9471,CWF - SANTANDER INDICAÇÃO SUBST,02-08-2018 09:29:57,pelo próprio candidato,sr._anthony_gomes@hotmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31001,formação acadêmica\nensino médio (2º grau) em ...,,,(24) 96341-6187,CONSULTORA ABAP SR,02-08-2018 10:45:46,pelo próprio candidato,dra._ana_sophia_guerra@gmail.com,,Outros,...,,,,,,,,,,
31002,objetivo: área administrativa | financeira\n\n...,,,(13) 98463-7528,~Consultor SAP MMWM,02-08-2018 11:20:33,pelo próprio candidato,srta._maria_cecília_garcia@gmail.com,"Santos, São Paulo",Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31003,formação\nensino médio completo\ninformática i...,,,011-9-8415-8195 97527-7215,Websphere/Middleware,02-08-2018 11:22:27,pelo próprio candidato,sra._melina_rios@hotmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31004,última atualização em 09/11/2021\n­ sp\n\nensi...,,,(11) 98856-4298,CONSULTOR ABAP SR,02-08-2018 11:41:34,pelo próprio candidato,yasmin_da_mota@gmail.com,,Outros,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,NaN,NaN,,(11) 95301-8362,Cobol,09-03-2018 15:19:05,pelo próprio candidato,fernando_mendonça@hotmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41996,NaN,NaN,,(11) 96151-6087,Cobol,09-03-2018 15:19:37,pelo próprio candidato,sr._nicolas_cavalcanti@hotmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41997,NaN,NaN,,(11) 96121-5143,Cobol,09-03-2018 15:21:02,pelo próprio candidato,dr._bruno_araújo@gmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41998,NaN,NaN,,(11) 99886-5245,Cobol Microfocus,09-03-2018 15:22:50,pelo próprio candidato,sr._luiz_miguel_fonseca@gmail.com,,Outros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# "Abrindo" os dados de applicants
df_prospects.reset_index(inplace=True)
df_prospects.rename(columns={'index': 'id_vaga'}, inplace=True)
df_prospects

,id_vaga,titulo,modalidade,prospects
0,4530,CONSULTOR CONTROL M,,"[{'nome': 'José Vieira', 'codigo': '25632', 's..."
1,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '..."
2,4532,,,[]
3,4533,2021-2605708-Microfocus Application Life Cycle...,,"[{'nome': 'Arthur Almeida', 'codigo': '26338',..."
4,4534,2021-2605711-Microfocus QTP - UFT Automation T...,,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361..."
...,...,...,...,...
14217,14218,Java Support Analyst,,"[{'nome': 'Dr. Vitor Hugo Silva', 'codigo': '4..."
14218,14219,,,[]
14219,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,"[{'nome': 'Ana Cardoso', 'codigo': '16828', 's..."
14220,14221,Consultor Sênior Oracle EPM FCCS - 1434,,"[{'nome': 'Maria Eduarda Cassiano', 'codigo': ..."


In [7]:
cols_com_dicionario_prospects =['prospects']

dfs_normalizados_prospects = []

for col in cols_com_dicionario_prospects:
    df_temp_prospects = pd.json_normalize(df_prospects[col])
    df_temp_prospects.columns = [f"{col}_{subcol}" for subcol in df_temp_prospects.columns]
    dfs_normalizados_prospects.append(df_temp_prospects)

df_normalizado_prospects = pd.concat([df_prospects.drop(columns=cols_com_dicionario_prospects)] + dfs_normalizados_prospects, axis=1)
df_normalizado_prospects

cols_com_dicionario_prospects_melted = ['dict_prospect']

df_normalizados_prospects_melted2 = []


In [8]:
df_normalizado_prospects_melted = df_normalizado_prospects.melt(
    id_vars=['titulo', 'modalidade','id_vaga'],
    var_name='n_prospect',
    value_name='dict_prospect'
)
df_normalizado_prospects_melted.sort_values('id_vaga')


,titulo,modalidade,id_vaga,n_prospect,dict_prospect
202750,,,0,prospects_14,None
273860,,,0,prospects_19,None
344970,,,0,prospects_24,None
330748,,,0,prospects_23,None
131640,,,0,prospects_9,None
...,...,...,...,...,...
241773,,,14222,prospects_16,None
312883,,,14222,prospects_21,None
127997,,,14222,prospects_8,None
56887,,,14222,prospects_3,None


In [9]:
cols_com_dicionario_prospects_melted = ['dict_prospect']

df_normalizados_prospects_melted2 = []

for col in cols_com_dicionario_prospects_melted:
    df_temp_prospects_melted = pd.json_normalize(df_normalizado_prospects_melted[col])
    df_temp_prospects_melted.columns = [f"{col}_{subcol}" for subcol in df_temp_prospects_melted.columns]
    df_normalizados_prospects_melted2.append(df_temp_prospects_melted)
    
dfs_normalizados_prospects_melted2 = pd.concat([df_normalizado_prospects_melted.drop(columns=cols_com_dicionario_prospects_melted)] + df_normalizados_prospects_melted2, axis=1)
dfs_normalizados_prospects_melted2

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador
0,CONSULTOR CONTROL M,,4530,prospects_0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,2021-2607395-PeopleSoft Application Engine-Dom...,,4531,prospects_0,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
2,,,4532,prospects_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-2605708-Microfocus Application Life Cycle...,,4533,prospects_0,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira
4,2021-2605711-Microfocus QTP - UFT Automation T...,,4534,prospects_0,Ana Luiza Vieira,26361,Documentação PJ,28-04-2021,11-05-2021,Aguardando confirmação de inicio _,Manuella Carvalho
...,...,...,...,...,...,...,...,...,...,...,...
355545,Java Support Analyst,,14218,prospects_24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355546,,,14219,prospects_24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355547,Consultor Sênior Especialista SAP LES-TRA - 1433,,14220,prospects_24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355548,Consultor Sênior Oracle EPM FCCS - 1434,,14221,prospects_24,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
colunas_comuns = set(df_normalizado.columns).intersection(dfs_normalizados_prospects_melted2.columns)
print(colunas_comuns)

set()


In [11]:
# Verificando uma linha aleatória
df_normalizado[df_normalizado['infos_basicas_codigo_profissional'] == '41659']

,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
1056,,,,(11) 92113-9734,Analista de Sistemas,13-02-2023 14:23:00,Emanuelly Vieira,sr._yago_cassiano@gmail.com,"São Paulo, São Paulo",,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
dfs_normalizados_prospects_melted2[dfs_normalizados_prospects_melted2['dict_prospect_codigo'] == '41659']

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador
11317,Analista de Desenvolvimento de Sistemas Pl/Sr-...,,11028,prospects_0,Sr. Yago Cassiano,41659,Encaminhado ao Requisitante,13-02-2023,13-02-2023,,Emanuelly Vieira


In [13]:
# As bases de 'df_normalizado' e 'dfs_normalizados_prospects_melted2' se cruzam pelas colunas 'cargo_atual_id_ibrati' e 'dict_prospect_codigo', respectivamente.
# Segue exemplos de verificação
## dfs_normalizados_prospects_melted2[dfs_normalizados_prospects_melted2['dict_prospect_codigo'] == '41659']
## df_normalizado[df_normalizado['infos_basicas_codigo_profissional'] == '41659']

In [14]:
# Analisando as tabelas de "vagas"

pd.set_option('display.max_columns', None)
cols_com_dicionario_df_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']

df_vagas_normalizado = []

for col in cols_com_dicionario_df_vagas:
    df_temp_vagas = pd.json_normalize(df_vagas[col])
    df_temp_vagas.columns = [f"{col}_{subcol}" for subcol in df_temp_vagas.columns]
    df_vagas_normalizado.append(df_temp_vagas)
    
df_vagas_normalizado = pd.concat([df_vagas.drop(columns=cols_com_dicionario_df_vagas)] + df_vagas_normalizado, axis=1)
df_vagas_normalizado

,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2
5185,23-11-2021,00-00-0000,Oracle - 12447747,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Luna Correia,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,07-12-2021,06-12-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Avançado,Básico,,TI - Projetos-,"Desenhar, construir e configurar aplicações pa...",Habilidade principal: Aplicativos Oracle PLSQL...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5184,23-11-2021,00-00-0000,E2E Consultant Copy 01 - 12447976,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Emanuella Campos,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,08-12-2021,07-04-2022,NaN,Brasil,São Paulo,São Paulo,Chácara Santo Antônio,Sul,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Projeto de Arquitetura - Revise e integre todo...,Outros detalhes do trabalho: 1 - SAS Campaign ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5183,23-11-2021,00-00-0000,Oracle - 12448269,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Laura Pacheco,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,29-11-2021,28-05-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Intermediário,Básico,,TI - Projetos-,"Oracle Financials Cloud - Projete, crie e conf...",Outros detalhes do trabalho: 1 - Oracle Financ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5182,23-11-2021,00-00-0000,Oracle - 12448277,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,07-12-2021,06-12-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,"Desenhar, construir e configurar aplicações pa...",Habilidades Primárias: Aplicativos Oracle PLSQ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5181,23-11-2021,00-00-0000,4621328 - Scrum Master,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sara Porto,Sra. Joana Sousa,CLT Full,Indeterminado,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Nenhum,Ne

In [15]:
# Primeiras conculsões
## A base de "applicants" são os candidatos que se inscreveram para as vagas
### Cruzando as colunas 'cargo_atual_id_ibrati' e 'dict_prospect_codigo' podemos encontrar os candidatos que estão concorrendo para uma vaga específica.
## A base de "prospects" são os candidatos que estão concorrendo para alguma vaga
## A base de "vagas" são as vagas disponíveis.

In [16]:
# Tratando os dados de "vagas" para análise

# Analisando as tabelas de "vagas"

df_vagas.reset_index(inplace=True)
df_vagas.rename(columns={'index': 'id_vaga'}, inplace=True)
df_vagas


,id_vaga,informacoes_basicas,perfil_vaga,beneficios
0,5185,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
1,5184,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
2,5183,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
3,5182,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '- p/ mês (168h)', 'valor_comp..."
4,5181,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
...,...,...,...,...
14076,12368,"{'data_requicisao': '28-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '123,00 -', 'valor_compra_1': ..."
14077,12367,"{'data_requicisao': '28-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."
14078,12366,"{'data_requicisao': '27-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."
14079,12365,"{'data_requicisao': '27-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."


In [17]:
#pd.set_option('display.max_columns', None)
cols_com_dicionario_df_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']

df_vagas_normalizado = []

for col in cols_com_dicionario_df_vagas:
    df_temp_vagas = pd.json_normalize(df_vagas[col])
    df_temp_vagas.columns = [f"{col}_{subcol}" for subcol in df_temp_vagas.columns]
    df_vagas_normalizado.append(df_temp_vagas)
    
df_vagas_normalizado = pd.concat([df_vagas.drop(columns=cols_com_dicionario_df_vagas)] + df_vagas_normalizado, axis=1)
df_vagas_normalizado

,id_vaga,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,18-05-2021,17-01-2022,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,
...,...,...,..

In [18]:
# Transformando os 3 dataframes em um só

df_final = pd.merge(
    dfs_normalizados_prospects_melted2,  # DataFrame da esquerda (o principal)
    df_vagas_normalizado,                # DataFrame da direita (de onde vêm as novas colunas)
    on='id_vaga',                        # Coluna usada como chave para a junção
    how='left'                           # Tipo de junção
)

df_normalizado = df_normalizado.dropna(subset=['infos_basicas_codigo_profissional'])

df_final_total = pd.merge(
    df_final,  # DataFrame da esquerda (o principal)
    df_normalizado, # DataFrame da direita (de onde vêm as novas colunas)
    left_on='dict_prospect_codigo',                        # Coluna usada como chave para a junção
    right_on='infos_basicas_codigo_profissional',
    how='left'                           # Tipo de junção
)

df_final_total

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,CONSULTOR CONTROL M,,4530,prospects_0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ P

In [19]:
# Criando as feature engeneering
# Para aprimorar o modelo, criamos alguns matches, como, inglÊs, espanhol, academico e das descrições

## Match linguagem (inglês e espanhol)

# Passo 1: Criar o Mapa de Níveis Ordenados (o mesmo para ambos os idiomas)
mapa_niveis = {
    'Nenhum': 0,
    '': 0,
    np.nan: 0,
    'Básico': 1,
    'Intermediário': 2,
    'Técnico': 2,
    'Avançado': 3,
    'Fluente': 4
}

# --- Processamento para o Inglês ---

# Passo 2a: Converter as colunas de Inglês para números
nivel_candidato_ingles_num = df_final_total['formacao_e_idiomas_nivel_ingles'].map(mapa_niveis).fillna(0)
nivel_vaga_ingles_num = df_final_total['perfil_vaga_nivel_ingles'].map(mapa_niveis).fillna(0)

# Passo 3a: Criar a coluna 'match_ingles'
df_final_total['match_ingles'] = (nivel_candidato_ingles_num >= nivel_vaga_ingles_num).astype(int)

# --- Novas linhas para o Espanhol ---

# Passo 2b: Converter as colunas de Espanhol para números
nivel_candidato_espanhol_num = df_final_total['formacao_e_idiomas_nivel_espanhol'].map(mapa_niveis).fillna(0)
nivel_vaga_espanhol_num = df_final_total['perfil_vaga_nivel_espanhol'].map(mapa_niveis).fillna(0)

# Passo 3b: Criar a coluna 'match_espanhol'
df_final_total['match_espanhol'] = (nivel_candidato_espanhol_num >= nivel_vaga_espanhol_num).astype(int)

# --- FIM DA SOLUÇÃO ---

print("\n--- DataFrame Final com as Novas Colunas de Match ---")
df_final_total


--- DataFrame Final com as Novas Colunas de Match ---


,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol
0,CONSULTOR CONTROL M,,4530,prospects_0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos d

In [20]:
## match academico

# Passo 1: Criar o Mapa de Níveis Acadêmicos Ordenados
mapa_academico = {
    # Nível 0 - Nulos e Vazios
    np.nan: 0,
    '': 0,
    
    # Nível 1 a 3 - Fundamental
    'Ensino Fundamental Incompleto': 1,
    'Ensino Fundamental Cursando': 2,
    'Ensino Fundamental Completo': 3,

    # Nível 4 a 6 - Médio
    'Ensino Médio Incompleto': 4,
    'Ensino Médio Cursando': 5,
    'Ensino Médio Completo': 6,

    # Nível 7 a 9 - Técnico
    'Ensino Técnico Incompleto': 7,
    'Ensino Técnico Cursando': 8,
    'Ensino Técnico Completo': 9,

    # Nível 10 a 12 - Superior
    'Ensino Superior Incompleto': 10,
    'Ensino Superior Cursando': 11,
    'Ensino Superior Completo': 12,

    # Nível 13 a 15 - Pós-Graduação
    'Pós Graduação Incompleto': 13,
    'Pós Graduação Cursando': 14,
    'Pós Graduação Completo': 15,

    # Nível 16 a 18 - Mestrado
    'Mestrado Incompleto': 16,
    'Mestrado Cursando': 17,
    'Mestrado Completo': 18,

    # Nível 19 a 21 - Doutorado
    'Doutorado Incompleto': 19,
    'Doutorado Cursando': 20,
    'Doutorado Completo': 21
}


# Passo 2: Converter as colunas de texto para números usando o mapa
nivel_candidato_num = df_final_total['formacao_e_idiomas_nivel_academico'].map(mapa_academico).fillna(0)
nivel_vaga_num = df_final_total['perfil_vaga_nivel_academico'].map(mapa_academico).fillna(0)

# Passo 3: Comparar os níveis e criar a coluna 'match_academico'
df_final_total['match_academico'] = (nivel_candidato_num >= nivel_vaga_num).astype(int)

print("\n--- DataFrame Final com a Nova Coluna 'match_academico' ---")
df_final_total


--- DataFrame Final com a Nova Coluna 'match_academico' ---


,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol,match_academico
0,CONSULTOR CONTROL M,,4530,prospects_0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprova

In [21]:
# Tratando a coluna 'informacoes_basicas_tipo_contratacao'

df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].fillna('').astype(str)
df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].str.replace(', ', ',') # Garante que não haja espaço após a vírgula
df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].str.strip() # Remove espaços no início/fim

In [22]:
# Realizando o label encoding na coluna no df_final_total 

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_tipo_contratacao'])[0]
df_final_total['informacoes_basicas_tipo_contratacao_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_prazo_contratacao'])[0]
df_final_total['informacoes_basicas_prazo_contratacao_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_prioridade_vaga'])[0]
df_final_total['informacoes_basicas_prioridade_vaga_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_origem_vaga'])[0]
df_final_total['informacoes_basicas_origem_vaga_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_estado'])[0]
df_final_total['perfil_vaga_estado_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel profissional'])[0]
df_final_total['perfil_vaga_nivel profissional_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_academico'])[0]
df_final_total['perfil_vaga_nivel profissional_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_ingles'])[0]
df_final_total['perfil_vaga_nivel_ingles_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_espanhol'])[0]
df_final_total['perfil_vaga_nivel_espanhol_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_viagens_requeridas'])[0]
df_final_total['perfil_vaga_viagens_requeridas_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_academico'])[0]
df_final_total['formacao_e_idiomas_nivel_academico_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_ingles'])[0]
df_final_total['formacao_e_idiomas_nivel_ingles_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_espanhol'])[0]
df_final_total['formacao_e_idiomas_nivel_espanhol_codificado'] = nova_coluna_codificada

In [23]:
# Realizando o one-hot encoding na coluna 'perfil_vaga_areas_atuacao' e 'informacoes_profissionais_area_atuacao'

## 'perfil_vaga_areas_atuacao'
### PADRONIZAR OS SEPARADORES

def limpar_e_padronizar_separadores(texto):
    # Retorna uma string vazia se o texto for nulo/inválido
    if pd.isna(texto):
        return ""
    
    # Converte para string para garantir
    texto_str = str(texto)
    
    # Regra 1: Remove o '-' solto no final da string
    texto_limpo = re.sub(r'-$', '', texto_str.strip())
    
    # Regra 2: Substitui o '-' que está entre dois caracteres por ', '
    texto_limpo = re.sub(r'(\S)-(\S)', r'\1, \2', texto_limpo)
    
    return texto_limpo

### Aplicar a função e o one hot encoding

df_final_total['areas_atuacao_limpas'] = df_final_total['perfil_vaga_areas_atuacao'].apply(limpar_e_padronizar_separadores)
df_areas_codificadas = df_final_total['areas_atuacao_limpas'].str.get_dummies(sep=', ')
prefixo = 'perfil_vaga_areas_atuacao' + '_'
df_areas_codificadas = df_areas_codificadas.add_prefix(prefixo)
df_final_total = pd.concat([df_final_total, df_areas_codificadas], axis=1)

## 'informacoes_profissionais_area_atuacao'

df_final_temporario = df_final_total['informacoes_profissionais_area_atuacao'].str.get_dummies(sep=', ')
prefixo = 'informacoes_profissionais_area_atuacao' + '_'
df_final_temporario = df_final_temporario.add_prefix(prefixo)
df_final_total = pd.concat([df_final_total, df_final_temporario], axis=1)

print("--- DataFrame Final Completo ---")
df_final_total

#df_final_total_backup = df_final_total.copy()

--- DataFrame Final Completo ---


,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol,match_academico,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,areas_atuacao_limpas,perfil_vaga_areas_atuacao_Administrativa,perfi

In [24]:
# 1. Defina a função que calcula o match 
def calcular_match_profissional(row):
    str_vaga = row['areas_atuacao_limpas']
    str_candidato = row['informacoes_profissionais_area_atuacao']

    # Verifica se o valor é nulo e o transforma em string vazia se for o caso.
    if pd.isna(str_vaga):
        str_vaga = ''
    if pd.isna(str_candidato):
        str_candidato = ''

    set_vaga = set(item.strip() for item in str_vaga.split(',') if item.strip())
    set_candidato = set(item.strip() for item in str_candidato.split(',') if item.strip())

    if not set_vaga:
        return pd.Series([0, 1.0], index=['match_areas_contagem', 'match_areas_percentual'])

    competencias_em_comum = set_vaga.intersection(set_candidato)
    
    match_contagem = len(competencias_em_comum)
    match_percentual = match_contagem / len(set_vaga)
    
    return pd.Series([match_contagem, match_percentual], index=['match_areas_contagem', 'match_areas_percentual'])


# 2. Aplique a função a cada linha do DataFrame
match_features = df_final_total.apply(calcular_match_profissional, axis=1)

# 3. Junte as novas features ao seu DataFrame principal
df_final_total = pd.concat([df_final_total, match_features], axis=1)


In [25]:
df_final_total

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol,match_academico,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,areas_atuacao_limpas,perfil_vaga_areas_atuacao_Administrativa,perfi

In [26]:
# Tratando a coluna 'dict_prospect_status'

## Adicionando a coluna 'status_geral'
mapeamento_status_ajustado = {
    # ## Categoria Sucesso
    'Aprovado': 'Sucesso',
    'Contratado como Hunting': 'Sucesso',
    'Contratado pela Decision': 'Sucesso',
    'Documentação Cooperado': 'Sucesso',
    'Documentação CLT': 'Sucesso',
    'Documentação PJ': 'Sucesso',
    'Proposta Aceita': 'Sucesso',

    # ## Categoria Insucesso (agora inclui as desistências)
    'Não Aprovado pelo RH': 'Insucesso',
    'Não Aprovado pelo Cliente': 'Insucesso',
    'Não Aprovado pelo Requisitante': 'Insucesso',
    'Recusado': 'Insucesso',
    'Desistiu da Contratação': 'Insucesso', 
    'Desistiu': 'Insucesso',                
    'Sem interesse nesta vaga': 'Insucesso',

    # ## Categoria Andamento
    'Encaminhado ao Requisitante': 'Andamento',
    'Inscrito': 'Andamento',
    'Prospect': 'Andamento',
    'Entrevista Técnica': 'Andamento',
    'Em avaliação pelo RH': 'Andamento',
    'Entrevista com Cliente': 'Andamento',
    'Encaminhar Proposta': 'Andamento'
}

df_final_total['status_geral'] = df_final_total['dict_prospect_situacao_candidado'].map(mapeamento_status_ajustado)

## Removendo linhas com status 'Andamento' e NaN na coluna 'status_geral'
df_final_total = df_final_total.dropna(subset=['status_geral'])
condicao_para_manter = df_final_total['status_geral'] != 'Andamento'
df_final_total = df_final_total[condicao_para_manter]

## Realizando o label encoding na coluna 'status_geral'
nova_coluna_codificada = pd.factorize(df_final_total['status_geral'])[0]
df_final_total['status_geral_codificado'] = nova_coluna_codificada 

print("DataFrame Após o Mapeamento com a Nova Lógica:")
df_final_total
print("-" * 50)
df_final_total

DataFrame Após o Mapeamento com a Nova Lógica:
--------------------------------------------------


,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol,match_academico,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,areas_atuacao_limpas,perfil_vaga_areas_atuacao_Administrativa,perfi

In [27]:
df_final_total

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_ingles,match_espanhol,match_academico,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,areas_atuacao_limpas,perfil_vaga_areas_atuacao_Administrativa,perfi

In [28]:
# Selecionando apenas as colunas para alimentar o modelo de ML

colunas_para_manter = [
    'id_vaga',
    'dict_prospect_codigo',
    'match_academico',
    'match_ingles',
    'match_espanhol',
    'match_areas_contagem',	
    'match_areas_percentual',
    'informacoes_basicas_tipo_contratacao_codificada',
    'informacoes_basicas_prazo_contratacao_codificada',
    'informacoes_basicas_prioridade_vaga_codificada',
    'informacoes_basicas_origem_vaga_codificada',
    'perfil_vaga_estado_codificada',
    'perfil_vaga_nivel profissional_codificado', # Atenção a este nome com espaço
    'perfil_vaga_nivel_ingles_codificado',
    'perfil_vaga_nivel_espanhol_codificado',
    'perfil_vaga_viagens_requeridas_codificado',
    'formacao_e_idiomas_nivel_academico_codificado',
    'formacao_e_idiomas_nivel_ingles_codificado',
    'formacao_e_idiomas_nivel_espanhol_codificado',
    'perfil_vaga_areas_atuacao_Administrativa',
    'perfil_vaga_areas_atuacao_Comercial',
    'perfil_vaga_areas_atuacao_Financeira/Controladoria',
    'perfil_vaga_areas_atuacao_Gestão e Alocação de Recursos de TI',
    'perfil_vaga_areas_atuacao_Jurídica',
    'perfil_vaga_areas_atuacao_Marketing',
    'perfil_vaga_areas_atuacao_Novos Negócios e Parcerias',
    'perfil_vaga_areas_atuacao_Qualidade Corporativa',
    'perfil_vaga_areas_atuacao_Recursos Humanos',
    'perfil_vaga_areas_atuacao_Relacionamento Técnico',
    'perfil_vaga_areas_atuacao_TI - Arquitetura',
    'perfil_vaga_areas_atuacao_TI - Banco de Dados',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Design',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Mobile',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Programação',
    'perfil_vaga_areas_atuacao_TI - Governança',
    'perfil_vaga_areas_atuacao_TI - Infraestrutura',
    'perfil_vaga_areas_atuacao_TI - Processos e Negócios',
    'perfil_vaga_areas_atuacao_TI - Projetos',
    'perfil_vaga_areas_atuacao_TI - Qualidade/Testes',
    'perfil_vaga_areas_atuacao_TI - SAP',
    'perfil_vaga_areas_atuacao_TI - Segurança da Informação',
    'perfil_vaga_areas_atuacao_TI - Sistemas e Ferramentas',
    'perfil_vaga_areas_atuacao_TI - Suporte',
    'perfil_vaga_areas_atuacao_TI - Telecom',
    'perfil_vaga_areas_atuacao_TI - UX',
    'informacoes_profissionais_area_atuacao_Administrativa',
    'informacoes_profissionais_area_atuacao_Comercial',
    'informacoes_profissionais_area_atuacao_Financeira/Controladoria',
    'informacoes_profissionais_area_atuacao_Gestão e Alocação de Recursos de TI',
    'informacoes_profissionais_area_atuacao_Jurídica',
    'informacoes_profissionais_area_atuacao_Marketing',
    'informacoes_profissionais_area_atuacao_Novos Negócios e Parcerias',
    'informacoes_profissionais_area_atuacao_Qualidade Corporativa',
    'informacoes_profissionais_area_atuacao_Recursos Humanos',
    'informacoes_profissionais_area_atuacao_Relacionamento Técnico',
    'informacoes_profissionais_area_atuacao_TI - Arquitetura',
    'informacoes_profissionais_area_atuacao_TI - Banco de Dados',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Design',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Mobile',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Programação',
    'informacoes_profissionais_area_atuacao_TI - Governança',
    'informacoes_profissionais_area_atuacao_TI - Infraestrutura',
    'informacoes_profissionais_area_atuacao_TI - Processos e Negócios',
    'informacoes_profissionais_area_atuacao_TI - Projetos',
    'informacoes_profissionais_area_atuacao_TI - Qualidade/Testes',
    'informacoes_profissionais_area_atuacao_TI - SAP',
    'informacoes_profissionais_area_atuacao_TI - Segurança da Informação',
    'informacoes_profissionais_area_atuacao_TI - Sistemas e Ferramentas',
    'informacoes_profissionais_area_atuacao_TI - Suporte',
    'informacoes_profissionais_area_atuacao_TI - Telecom',
    'informacoes_profissionais_area_atuacao_TI - UX',
    'status_geral_codificado'
]

df_final_total = df_final_total[colunas_para_manter]
df_final_total

,id_vaga,dict_prospect_codigo,match_academico,match_ingles,match_espanhol,match_areas_contagem,match_areas_percentual,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,perfil_vaga_areas_atuacao_Administrativa,perfil_vaga_areas_atuacao_Comercial,perfil_vaga_areas_atuacao_Financeira/Controladoria,perfil_vaga_areas_atuacao_Gestão e Alocação de Recursos de TI,perfil_vaga_areas_atuacao_Jurídica,perfil_vaga_areas_atuacao_Marketing,perfil_vaga_areas_atuacao_Novos Negócios e Parcerias,perfil_vaga_areas_atuacao_Qualidade Corporativa,perfil_vaga_areas_atuacao_Recursos Humanos,perfil_vaga_areas_atuacao_Relacionamento Técnico,perfil_vaga_areas_atuacao_TI - Arquitetura,perfil_vaga_areas_atuacao_TI - Banco de Dados,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Design,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Mobile,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Programação,perfil_vaga_areas_atuacao_TI - Governança,perfil_vaga_areas_atuacao_TI - Infraestrutura,perfil_vaga_areas_atuacao_TI - Processos e Negócios,perfil_vaga_areas_atuacao_TI - Projetos,perfil_vaga_areas_atuacao_TI - Qualidade/Testes,perfil_vaga_areas_atuacao_TI - SAP,perfil_vaga_areas_atuacao_TI - Segurança da Informação,perfil_vaga_areas_atuacao_TI - Sistemas e Ferramentas,perfil_vaga_areas_atuacao_TI - Suporte,perfil_vaga_areas_atuacao_TI - Telecom,perfil_vaga_areas_atuacao_TI - UX,informacoes_profissionais_area_atuacao_Administrativa,informacoes_profissionais_area_atuacao_Comercial,informacoes_profissionais_area_atuacao_Financeira/Controladoria,informacoes_profissionais_area_atuacao_Gestão e Alocação de Recursos de TI,informacoes_profissionais_area_atuacao_Jurídica,informacoes_profissionais_area_atuacao_Marketing,informacoes_profissionais_area_atuacao_Novos Negócios e Parcerias,informacoes_profissionais_area_atuacao_Qualidade Corporativa,informacoes_profissionais_area_atuacao_Recursos Humanos,informacoes_profissionais_area_atuacao_Relacionamento Técnico,informacoes_profissionais_area_atuacao_TI - Arquitetura,informacoes_profissionais_area_atuacao_TI - Banco de Dados,informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Design,informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Mobile,informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Programação,informacoes_profissionais_area_atuacao_TI - Governança,informacoes_profissionais_area_atuacao_TI - Infraestrutura,informacoes_profissionais_area_atuacao_TI - Processos e Negócios,informacoes_profissionais_area_atuacao_TI - Projetos,informacoes_profissionais_area_atuacao_TI - Qualidade/Testes,informacoes_profissionais_area_atuacao_TI - SAP,informacoes_profissionais_area_atuacao_TI - Segurança da Informação,informacoes_profissionais_area_atuacao_TI - Sistemas e Ferramentas,informacoes_profissionais_area_atuacao_TI - Suporte,informacoes_profissionais_area_atuacao_TI - Telecom,informacoes_profissionais_area_atuacao_TI - UX,status_geral_codificado
1,4531,25364,1,1,1,0.0,0.0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,4533,26338,0,0,0,0.0,0.0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4534,26361,0,0,1,0.0,0.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,4535,26338,0,0,1,0.0,0.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
# Testando um modelo de base (base line model)
# O modelo escolhido como base line é o Random Forest Classifier

# A variável 'y' é a coluna que queremos prever (o nosso alvo).
y = df_final_total['status_geral_codificado']

colunas_para_remover = ['status_geral_codificado', 'id_vaga', 'dict_prospect_codigo']
X = df_final_total.drop(columns=colunas_para_remover)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utilizar o smote para equilibrar as classes

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Vamos imprimir o "formato" (shape) de cada conjunto de dados para confirmar a divisão.
print("Verificação da Divisão dos Dados:")
print(f"Formato de X_train (features de treino): {X_train.shape}")
print(f"Formato de y_train (alvo de treino): {y_train.shape}")
print(f"Formato de X_test (features de teste): {X_test.shape}")
print(f"Formato de y_test (alvo de teste): {y_test.shape}")

print("\nPreparação concluída! Os dados estão prontos para o treinamento do modelo.")

Verificação da Divisão dos Dados:
Formato de X_train (features de treino): (14422, 69)
Formato de y_train (alvo de treino): (14422,)
Formato de X_test (features de teste): (2443, 69)
Formato de y_test (alvo de teste): (2443,)

Preparação concluída! Os dados estão prontos para o treinamento do modelo.


In [30]:
print("Iniciando o treinamento do modelo Random Forest...")

# Criamos uma instância do classificador. 
modelo_rf = RandomForestClassifier(n_estimators=150, 
                                   random_state=42)
                                   

# O método .fit() é onde o modelo "aprende" com os dados de treino.
modelo_rf.fit(X_train, y_train)

print("Treinamento concluído!")

# Usamos o modelo treinado para prever os resultados dos dados de teste.
y_pred = modelo_rf.predict(X_test)

# --- PASSO 5: Avaliar a Performance do Modelo ---
print("\n--- Avaliação do Modelo no Conjunto de Teste ---")

# 1. Acurácia
# A métrica mais simples: porcentagem de acertos.
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {acuracia:.2%}")

Iniciando o treinamento do modelo Random Forest...
Treinamento concluído!

--- Avaliação do Modelo no Conjunto de Teste ---
Acurácia do modelo: 76.09%


In [31]:
# 3. Relatório de Classificação

print("\nRelatório de Classificação:")
report = classification_report(y_test, y_pred)
print(report)


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.55      0.58      0.56       646
           1       0.84      0.83      0.84      1797

    accuracy                           0.76      2443
   macro avg       0.69      0.70      0.70      2443
weighted avg       0.77      0.76      0.76      2443



In [32]:
# --- PASSO 6: Analisar a Importância das Features ---
print("\n--- Features Mais Importantes para o Modelo ---")

# Criamos uma Série do Pandas para visualizar melhor as importâncias
importancias = pd.Series(modelo_rf.feature_importances_, index=X.columns)

# Ordenamos da mais importante para a menos importante e mostramos as top 15
importancias_ordenadas = importancias.sort_values(ascending=False)
print(importancias_ordenadas.head(30))


--- Features Mais Importantes para o Modelo ---
informacoes_basicas_tipo_contratacao_codificada                            0.152427
perfil_vaga_nivel_ingles_codificado                                        0.103790
informacoes_basicas_prazo_contratacao_codificada                           0.059320
perfil_vaga_estado_codificada                                              0.054236
informacoes_basicas_origem_vaga_codificada                                 0.053116
perfil_vaga_nivel profissional_codificado                                  0.050653
informacoes_basicas_prioridade_vaga_codificada                             0.044013
perfil_vaga_areas_atuacao_TI - Projetos                                    0.043406
perfil_vaga_nivel_espanhol_codificado                                      0.038347
perfil_vaga_viagens_requeridas_codificado                                  0.035801
formacao_e_idiomas_nivel_academico_codificado                              0.031084
formacao_e_idiomas_nivel_es

In [33]:
# Testando o hiperparâmetro n_estimators

# ----------------------------- LOOP DE TESTE DOS PARAMETROS DO MODELO -----------------------------
# # 1. Crie uma lista de valores que queremos testar para n_estimators
# n_estimators_lista = list(range(10, 201, 10))

# # 2. Crie uma lista vazia para guardar os resultados
# resultados_f1 = []

# print("Iniciando análise do parâmetro n_estimators...")
# # 3. Crie um loop para treinar e avaliar o modelo para cada valor da lista
# for n in n_estimators_lista:
#     # Cria o modelo com o número de árvores atual (n)
#     modelo_rf = RandomForestClassifier(n_estimators=n, random_state=42)
    
#     # Treina o modelo com os dados balanceados
#     modelo_rf.fit(X_train, y_train)
    
#     # Faz as previsões
#     y_pred = modelo_rf.predict(X_test)
    
#     # Calcula o F1-Score e guarda na lista de resultados
#     # 'weighted' calcula a média ponderada pelo suporte de cada classe
#     score = f1_score(y_test, y_pred, average='weighted')
#     resultados_f1.append(score)
    
#     print(f"  n_estimators = {n}, F1-Score (Weighted) = {score:.4f}")

# print("Análise concluída!")
# ----------------------------- LOOP DE TESTE DOS PARAMETROS DO MODELO -----------------------------

# O resultado varia pouco, mas vou manter 150 que ficou levemente superior


In [34]:
# Testando o modelo com uma rede neural (Mult Layer Perceptron)

# --- PASSO 1: Escalonamento dos Dados ---
# Criamos o objeto scaler
scaler = StandardScaler()

# Ajuste o scaler APENAS nos dados de treino
X_train = scaler.fit_transform(X_train)

# Use o scaler já ajustado para TRANSFORMAR os dados de teste
X_test = scaler.transform(X_test)

# --- PASSO 2: Construção da Arquitetura do Modelo ---
# O número de features de entrada é o número de colunas em X
n_features = X_train.shape[1]

# Usamos o modelo Sequential, que nos permite empilhar camadas uma a uma
modelo_nn = Sequential()

# Camada de Entrada e Primeira Camada Oculta
modelo_nn.add(Dense(64, activation='relu', input_shape=(n_features,)))
modelo_nn.add(Dropout(0.4)), # Desliga 50% dos neurônios para combater overfitting

# Segunda Camada Oculta
modelo_nn.add(Dense(32, activation='relu'))
modelo_nn.add(Dropout(0.2)), # Desliga 30% dos neurônios

modelo_nn.add(Dense(16, activation='relu')), # <-- Nova camada

# Camada de Saída
modelo_nn.add(Dense(1, activation='sigmoid'))

# --- PASSO 3: Compilação do Modelo ---
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
modelo_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostra um resumo da arquitetura criada
print("--- Arquitetura da Rede Neural ---")
modelo_nn.summary()

#Após o teste no final do notebook, os parametros de omitização foram:

# neurons_1: 64
# neurons_2: 32
# dropoutrate: 0.4
# learning_rate: 0.0010



c:\Users\Usuário\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Arquitetura da Rede Neural ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,105 (27.75 KB)

 Trainable params: 7,105 (27.75 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Configura o callback de Parada Antecipada
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# --- PASSO 4: Treinamento do Modelo ---
print("\n--- Iniciando o Treinamento da Rede Neural ---")
historico = modelo_nn.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_test, y_test),
    verbose=1 # verbose=1 mostra uma barra de progresso
)




--- Iniciando o Treinamento da Rede Neural ---
Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5976 - loss: 0.6690 - val_accuracy: 0.7212 - val_loss: 0.5946
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6960 - loss: 0.5905 - val_accuracy: 0.7311 - val_loss: 0.5805
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7010 - loss: 0.5726 - val_accuracy: 0.7372 - val_loss: 0.5812
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7212 - loss: 0.5451 - val_accuracy: 0.7421 - val_loss: 0.5841
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7232 - loss: 0.5362 - val_accuracy: 0.7511 - val_loss: 0.5716
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7282 - loss: 0.5322 - val_accuracy: 0.7507 - val_loss: 0.5695
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7284 - loss: 0.5231 - val_accuracy: 0.7470 - val_loss: 0.5580
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/s

In [36]:
# --- PASSO 5: Avaliação Final do Modelo ---
print("\n--- Avaliação Final do Modelo nos Dados de Teste ---")

# .predict() retorna as probabilidades de pertencer à classe 1
probabilidades = modelo_nn.predict(X_test)
# Convertemos as probabilidades em classes (0 ou 1) usando um limiar de 0.5
y_pred_nn = (probabilidades > 0.5).astype(int)

print("\nRelatório de Classificação da Rede Neural:")
print(classification_report(y_test, y_pred_nn))


--- Avaliação Final do Modelo nos Dados de Teste ---
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Relatório de Classificação da Rede Neural:
              precision    recall  f1-score   support

           0       0.54      0.49      0.51       646
           1       0.82      0.85      0.84      1797

    accuracy                           0.76      2443
   macro avg       0.68      0.67      0.68      2443
weighted avg       0.75      0.76      0.75      2443



In [39]:
# 1. Obter as probabilidades usando o método .predict()

print("Gerando probabilidades com a Rede Neural...")
probabilidades_reprovado = modelo_nn.predict(X_test)

prob_ser_aprovado = 1 - probabilidades_reprovado.flatten()

# 3. Criar um novo DataFrame para o ranking
df_ranking = X_test.copy()

df_ranking = pd.DataFrame(X_test, columns=X.columns)

# 4. Adicionar as informações importantes para análise
df_ranking['probabilidade_match'] = prob_ser_aprovado
df_ranking['status_real'] = y_test # Adiciona o resultado real para comparação

# Para facilitar a identificação, vamos buscar os IDs originais usando o índice do DataFrame
df_ranking = df_ranking.join(df_final_total[['id_vaga', 'dict_prospect_codigo']])

# 5. Ordenar o ranking pela probabilidade, do maior para o menor
df_ranking_final = df_ranking.sort_values(by='probabilidade_match', ascending=False)
df_ranking_final = df_ranking_final[df_ranking_final['id_vaga'].notna()]

# 6. Exibir o resultado
print("\n--- Ranking de Candidatos (Gerado pela Rede Neural) ---")
colunas_para_exibir = ['id_vaga', 'dict_prospect_codigo', 'probabilidade_match', 'status_real']
df_ranking_final[colunas_para_exibir].head(20)

Gerando probabilidades com a Rede Neural...
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

--- Ranking de Candidatos (Gerado pela Rede Neural) ---


,id_vaga,dict_prospect_codigo,probabilidade_match,status_real
1988,6062.0,22489,0.994754,NaN
1884,5958.0,28270,0.994754,NaN
178,4708.0,25613,0.989811,NaN
63,4593.0,25365,0.971902,NaN
855,8349.0,26909,0.971214,NaN
1028,8522.0,33996,0.970613,1.0
1847,5921.0,18774,0.970077,NaN
1326,5400.0,27230,0.962341,NaN
2377,6451.0,24330,0.958848,NaN
1794,5868.0,27950,0.958848,NaN


In [38]:
# --- Passos de preparação de dados (como antes) ---
# Configuramos algumas opções de combinações para buscar o melhor resultado do modelo. 
# Foi testado modelos similares ao gridsearch para buscar resultados melhores, mas não rodou bem o modelo, então fizemos "na força bruta" mesmo.

#-----------------LOOP DE TESTE DOS PARAMETROS DO MODELO-----------------
# --- PASSO 1: Função para criar o modelo ---
# A função precisa do n_features, que vamos definir logo em seguida
# def criar_modelo_nn(n_features, neurons_1=64, neurons_2=32, neurons_3=None, dropout_rate=0.5, learning_rate=0.001):
#     model = Sequential()
#     model.add(Dense(neurons_1, activation='relu', input_shape=(n_features,)))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(neurons_2, activation='relu'))
#     model.add(Dropout(dropout_rate / 2))
#     if neurons_3:
#         model.add(Dense(neurons_3, activation='relu'))
#         model.add(Dropout(dropout_rate / 2.5))
#     model.add(Dense(1, activation='sigmoid'))
    
#     optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # --- PASSO 2: Definir as combinações e o n_features ---
# n_features = X_train.shape[1]

# # Lista com 20 combinações estratégicas para testar
# combinacoes_de_parametros = [
#     # --- TEMA 1: Arquiteturas Padrão (64 -> 32) com variações ---
#     # A que já conhecemos e se saiu bem
#     {'neurons_1': 64, 'neurons_2': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001},
#     # Menos regularização (dropout menor)
#     {'neurons_1': 64, 'neurons_2': 32, 'dropout_rate': 0.3, 'learning_rate': 0.001},
#     # Taxa de aprendizado mais lenta
#     {'neurons_1': 64, 'neurons_2': 32, 'dropout_rate': 0.5, 'learning_rate': 0.0001},
    
#     # --- TEMA 2: Arquiteturas "Largas" (mais neurônios por camada) ---
#     # Uma versão mais larga da nossa melhor
#     {'neurons_1': 128, 'neurons_2': 64, 'dropout_rate': 0.5, 'learning_rate': 0.001},
#     # Mais larga e com menos dropout
#     {'neurons_1': 128, 'neurons_2': 64, 'dropout_rate': 0.3, 'learning_rate': 0.001},
#     # Ainda mais larga e com mais dropout para compensar
#     {'neurons_1': 256, 'neurons_2': 128, 'dropout_rate': 0.6, 'learning_rate': 0.001},
    
#     # --- TEMA 3: Arquiteturas "Profundas" (3 camadas ocultas) ---
#     # Nossa melhor arquitetura, mas com uma camada extra
#     {'neurons_1': 64, 'neurons_2': 32, 'neurons_3': 16, 'dropout_rate': 0.4, 'learning_rate': 0.001},
#     # Uma versão mais larga e profunda
#     {'neurons_1': 128, 'neurons_2': 64, 'neurons_3': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001},
#     # Uma versão profunda sem afunilar
#     {'neurons_1': 64, 'neurons_2': 64, 'neurons_3': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001},
    
#     # --- TEMA 4: Arquiteturas "Simples" ---
#     # Uma rede bem menor para ver se o problema não precisa de tanta complexidade
#     {'neurons_1': 32, 'neurons_2': 16, 'dropout_rate': 0.3, 'learning_rate': 0.001},
#     # Outra variação simples
#     {'neurons_1': 32, 'neurons_2': 32, 'dropout_rate': 0.4, 'learning_rate': 0.001},
    
#     # --- TEMA 5: Variações de Taxa de Aprendizado ---
#     # Testando uma taxa de aprendizado alta (pode divergir)
#     {'neurons_1': 64, 'neurons_2': 32, 'dropout_rate': 0.5, 'learning_rate': 0.01},
#     # Arquitetura larga com aprendizado lento
#     {'neurons_1': 128, 'neurons_2': 64, 'dropout_rate': 0.5, 'learning_rate': 0.0001},
#     # Arquitetura profunda com aprendizado lento
#     {'neurons_1': 64, 'neurons_2': 32, 'neurons_3': 16, 'dropout_rate': 0.5, 'learning_rate': 0.0001},

#     # --- TEMA 6: Combinações Adicionais para Explorar ---
#     # Rede larga, mas afunilando rapidamente
#     {'neurons_1': 128, 'neurons_2': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001},
#     # Rede muito larga com regularização muito forte
#     {'neurons_1': 256, 'neurons_2': 64, 'dropout_rate': 0.7, 'learning_rate': 0.001},
#     # Arquitetura média com aprendizado lento e pouco dropout
#     {'neurons_1': 64, 'neurons_2': 64, 'dropout_rate': 0.2, 'learning_rate': 0.0001},
#     # Variação da nossa melhor, com dropout um pouco menor
#     {'neurons_1': 64, 'neurons_2': 32, 'dropout_rate': 0.4, 'learning_rate': 0.001},
#     # Uma arquitetura profunda e mais larga
#     {'neurons_1': 128, 'neurons_2': 128, 'neurons_3': 64, 'dropout_rate': 0.5, 'learning_rate': 0.001},
#     # Uma arquitetura muito simples com aprendizado lento
#     {'neurons_1': 32, 'neurons_2': 16, 'dropout_rate': 0.2, 'learning_rate': 0.0001}
# ]

# # --- PASSO 3: Loop de Treinamento e Avaliação Manual (CORRIGIDO) ---
# resultados = []
# callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# print(f"Iniciando a busca manual por {len(combinacoes_de_parametros)} combinações...")

# for i, params in enumerate(combinacoes_de_parametros):
#     print(f"\n--- Testando Combinação {i+1}/{len(combinacoes_de_parametros)} ---")
#     print(params)
    
#     # Cria um novo modelo, agora passando o n_features
#     # E também passando o neurons_3, que pode ser None
#     modelo = criar_modelo_nn(
#         n_features=n_features, # <-- CORREÇÃO 1
#         neurons_1=params.get('neurons_1', 64),
#         neurons_2=params.get('neurons_2', 32),
#         neurons_3=params.get('neurons_3', None), # .get() lida com chaves ausentes
#         dropout_rate=params.get('dropout_rate', 0.5),
#         learning_rate=params.get('learning_rate', 0.001)
#     )
    
#     # Treina o modelo com os dados corretos: balanceados e escalonados
#     modelo.fit(
#         X_train,      
#         y_train,   
#         epochs=100,
#         batch_size=32,
#         validation_data=(X_test, y_test), 
#         callbacks=[callback_early_stopping],
#         verbose=0
#     )
    
#     # Avalia o modelo nos dados de teste escalonados
#     y_pred_probs = modelo.predict(X_test) 
#     y_pred_classes = (y_pred_probs > 0.5).astype(int)
    
#     score = f1_score(y_test, y_pred_classes, average='weighted')
#     print(f"F1-Score (Weighted) para esta combinação: {score:.4f}")
    
#     resultado_atual = params.copy()
#     resultado_atual['f1_score'] = score
#     resultados.append(resultado_atual)

# # --- PASSO 4: Analisar os Resultados ---
# print("\n--- Busca Manual Concluída ---")
# df_resultados = pd.DataFrame(resultados)
# df_resultados_ordenados = df_resultados.sort_values(by='f1_score', ascending=False)

# print("Ranking de performance das combinações testadas:")
# print(df_resultados_ordenados)

#-----------------LOOP DE TESTE DOS PARAMETROS DO MODELO-----------------